**Created on**: 2022.03.04

**Implemented by**: Anthony Cho

**Subject**: Linear programming: Manufacturing Company

**Reference**: Introduction to operation research (10th Edition) - Hillier and Lieberman.
**Problem**: 3.1-11

## Problem: 

The Omega Manufacturing Company has discontinued the production of a certain unprofitable product line. This act created considerable excess production capacity. Management is considering devoting this excess capacity to one or more of three products; call them products 1, 2, and 3. The available capacity on the machines that might limit output is summarized in the following table:

| **Machine Type** | **Available Time (Machine Hours per Week)**      |
|------------------|--------------------------------------------------|
| Milling machine  | 500                                              |
| Lathe            | 350                                              |
| Grinder          | 150                                              |


The number of machine hours required for each unit of the respective products is

| **Machine Type** | **Product 1** | **Product 2** | **Product 3** |
|------------------|---------------|---------------|---------------|
| Milling machine  | 9             | 3             | 5             |
| Lathe            | 5             | 4             | 0             |
| Grinder          | 3             | 0             | 2             |


The sales department indicates that the sales potential for products 1 and 2 exceeds the maximum production rate and that the sales potential for product 3 is 20 units per week. 

The unit profit would be:

|        | **Product 1** | **Product 2** | **Product 3** |
|--------|---------------|---------------|---------------|
| Profit | &#36;50       | &#36;20       | &#36;25       |


The objective is to determine how much of each product Omega should produce to maximize profit.

**Parameters**:
* $J$: Set of products.
* $I$: Set of Machines.
* $g_j$: expected profit per unit by product $j$ ($j \in J$).
* $w_{i,j}$: amount of machine-hours per week required for each unit of product $j$ processed by the machine type $i$  ($i \in I, \; j \in J$).
* $l_i$: machine-hours per week available in machine type $i$ ($i \in I$).
* $b_k$: maximum number of product k to produce (if exist). 

**Decision variables**:
* $X_j$: Number of units of product $j$ to produce ($j \in J$).

**Constraints**:
* **Machine-hours per week limit**: $\quad \sum_{j \in J} w_{i,j} \cdot X_{j} \leq l_j, \quad \forall i \in I.$
* **Limit for product k**: $X_{k} \leq b_{k}, \quad \forall k \in K = \{j \in J| b_{j} \; exist\}$
* **Decision variable bound**: $X_{j} \geq 0, \quad \forall j$.

**Objective function**:
* $\max_{X} \sum_{j} g_{j} \cdot X_{j}$.


### General model

$$\max_{X} \sum_{j \in J} g_{j} \cdot X_{j}$$

s.t.

$$\quad \sum_{j} w_{i,j} \cdot X_{j} \leq l_j, \quad \forall i \in I.$$
$$X_{k} \leq b_{k}, \quad \forall k \in K = \{j \in J| b_{j} \; exist\}$$
$$X_{j} \geq 0, \quad \forall j \in J$$

### Particular model

$$\max_{X} 50 X_{1} + 20 X_{2} + 25 X_{3}$$

s.t.

|          |           |           |        |      |
|----------|-----------|-----------|--------|------|
| $9X_{1}$ | $+3X_{2}$ | $+5X_{3}$ | $\leq$ | 500  |
| $5X_{1}$ | $4X_{2}$  |           | $\leq$ | 350  |
| $3X_{1}$ |           | $+2X_{3}$ | $\leq$ | 150  |
|          |           | $ X_{3}$  | $\leq$ | 20   |


$$X_{1}, X_{2}, X_{3} \geq 0$$

In [1]:
## Libraries dependencies

import gurobipy as gp

### Parameters

In [2]:
## Expected profit per unit by product
profit = {'Product 1': 50, 
          'Product 2': 20,
          'Product 3': 25}

## Machine-hours per week required by product each
W = {'Milling machine': {'Product 1': 9, 'Product 2': 3, 'Product 3': 5},
     'Lathe': {'Product 1': 5, 'Product 2': 4, 'Product 3': 0},
     'Grinder': {'Product 1': 3, 'Product 2': 0, 'Product 3': 2}
    }

## Maximum units of each product to produce
B = {'Product 3': 20}

## Machie-hours available per machine type
L = {'Milling machine': 500,
     'Lathe': 350,
     'Grinder': 150
    }

### LP-Model

In [3]:
## Model instance
model = gp.Model('ManufacturingCompany')
model.modelSense = gp.GRB.MAXIMIZE

Academic license - for non-commercial use only - expires 2022-11-29
Using license file /home/hp/gurobi.lic


In [4]:
## Decision-variables
X = {}
for p in profit.keys():
    X[p] = model.addVar(obj=profit[p], lb=0, vtype=gp.GRB.CONTINUOUS, name=f'X[{p}]')

In [5]:
## Constraints
for machine in W.keys():
    lexp = gp.LinExpr()
    for p in W[machine].keys():
        lexp.addTerms(W[machine][p], X[p])
    model.addConstr(lexp, gp.GRB.LESS_EQUAL, L[machine], name=f'MaxMachineHours[{machine}]')
    
for p in B.keys():
    model.addConstr(X[p], gp.GRB.LESS_EQUAL, B[p], name=f'MaxProduct[{p}]')

In [6]:
## Model update
model.update()

In [7]:
## Model display
model.display()

Maximize
   <gurobi.LinExpr: 50.0 X[Product 1] + 20.0 X[Product 2] + 25.0 X[Product 3]>
Subject To
   MaxMachineHours[Milling machine] : <gurobi.LinExpr: 9.0 X[Product 1] + 3.0 X[Product 2] + 5.0 X[Product 3]> <= 500.0
   MaxMachineHours[Lathe] : <gurobi.LinExpr: 5.0 X[Product 1] + 4.0 X[Product 2]> <= 350.0
   MaxMachineHours[Grinder] : <gurobi.LinExpr: 3.0 X[Product 1] + 2.0 X[Product 3]> <= 150.0
   MaxProduct[Product 3] : <gurobi.LinExpr: X[Product 3]> <= 20.0


In [8]:
## Optimize model
model.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (linux64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 4 rows, 3 columns and 8 nonzeros
Model fingerprint: 0x5ff64e49
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [2e+01, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 5e+02]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 3 rows, 3 columns, 7 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.3333333e+03   3.956121e+01   0.000000e+00      0s
       2    2.9047619e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds
Optimal objective  2.904761905e+03


#### Post-Processing: Interpretation

In [9]:
if model.status == gp.GRB.OPTIMAL:
    for p in X.keys():
        if X[p].x:
            print('Produce {:>5.2f} units of {}'.format(X[p].x, p))

    print(f'\nTotal profit ${model.ObjVal:.3f}\n')
    
    for c in model.getConstrs():
        print('Slack {:40}: {:>8.2f}'.format(c.ConstrName, c.slack))
    
else:
    print('No solution was found.')

Produce 26.19 units of Product 1
Produce 54.76 units of Product 2
Produce 20.00 units of Product 3

Total profit $2904.762

Slack MaxMachineHours[Milling machine]        :     0.00
Slack MaxMachineHours[Lathe]                  :     0.00
Slack MaxMachineHours[Grinder]                :    31.43
Slack MaxProduct[Product 3]                   :     0.00
